In [207]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv("../../data/processed/data_2.csv",
                   dtype = {
                            "expensa_id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object",
                            "expensa_padre_id": "object",
                            "unidad_prop": "object",
                            "consorcio_id": "object",
                            "usuario_creador_id": "object",
                            "concepto_id": "object",
                            "mes": "object",
                            "anio": "object",
                            "unidad_denominacion":"object",
                            "consorcio_nombre": "object",
                            "version":"object",
                            "concepto_id": "object",
                            "unidad_tipo": "object",
                            "modo_pago": "object",
                            "forma_prorrateo": "object",
                            "tipo_prorrateo": "object",
                            "metodo_pago": "object",    
                            "venc_1_dayofweek": "object",
                            "venc_2_dayofweek": "object",
                            "venc_1_dayofweek_ant": "object",
                            "venc_2_dayofweek_ant": "object",
                            "dia_inicio_mes_vencimiento": "object"
                    },
                    parse_dates = ["fecha_vencimiento_1", "fecha_vencimiento_2"])

data = data[['unidad_id', 'unidad_tipo', 'modo_pago', 'tamanio_consorcio',
             'fecha_expensa_x','mes_vencimiento','dia_inicio_mes_vencimiento',
             'fecha_vencimiento_1', 'venc_1_dayofweek', 'venc_1_day',
             'fecha_vencimiento_2', 'venc_2_dayofweek', 'venc_2_day',             
             'monto', 'monto_total', 'monto_parcial', 
             'int_dia', 'int_mes', 
             'cum_impago_ant', 'cum_pago_tardio_1_ant','cum_pago_tardio_2_ant',
             'mes',  
             'version',
             'metodo_pago',
             'impago', 'pago_tardio_1', 'pago_tardio_2',
             'fecha_expensa_anterior']]

data.columns = ['unidad_id', 'unidad_tipo', 'modo_pago', 'tamanio_consorcio',
                 'fecha_expensa','mes_vencimiento','dia_inicio_mes_vencimiento',
                 'fecha_vencimiento_1', 'fecha_vencimiento_1_dayofweek', 'fecha_vencimiento_1_daynumber',
                 'fecha_vencimiento_2', 'fecha_vencimiento_2_dayofweek', 'fecha_vencimiento_2_daynumber',             
                 'monto', 'monto_total', 'monto_parcial', 
                 'int_dia', 'int_mes', 
                 'cantidad_impagos_a_la_fecha', 'cantidad_pagos_tardios_1_a_la_fecha','cantidad_pagos_tardios_2_a_la_fecha',
                 'mes',  
                 'version',        
                 'metodo_pago',
                 'impago', 'pago_tardio_1', 'pago_tardio_2',
                 'fecha_expensa_anterior']

data["target"] = data["impago"].map(int).map(str) + data["pago_tardio_1"].map(int).map(str) +data["pago_tardio_2"].map(int).map(str)

data.loc[data.target == '000', "target"] = 0
data.loc[data.target == '100', "target"] = 1
data.loc[data.target == '010', "target"] = 2
data.loc[data.target == '001', "target"] = 3
data = data.set_index(["unidad_id","fecha_expensa"])

In [208]:
data_mes_anterior = data[['target', 'fecha_expensa_anterior']]

In [209]:
data_lstm = data.reset_index()[["unidad_id","fecha_expensa","target", "fecha_expensa_anterior"]]

In [210]:
data_lstm = pd.merge(data_lstm,
         data_mes_anterior,
         left_on = ["unidad_id","fecha_expensa_anterior"],
         right_index = True,
         suffixes = ("", "_lag_1"))

for i in range(1,6):
    data_lstm = pd.merge(data_lstm,
                         data_mes_anterior,
                         left_on = ["unidad_id","fecha_expensa_anterior_lag_{}".format(i)],
                         right_index = True,
                         suffixes = ("", "_lag_{}".format(i+1)))

In [211]:
target_columns = ["fecha_expensa"] + [column for column in data_lstm.columns if column[0:6] == "target"]

In [212]:
data_lstm = data_lstm[target_columns]

In [213]:
data_lstm.shape

(213177, 8)

In [214]:
X = data_lstm.drop(columns="target")
Y = data_lstm[["fecha_expensa", "target"]]

In [215]:
X = X.loc[data_lstm.fecha_expensa < "2019-01-01",:]
Y = Y.loc[data_lstm.fecha_expensa < "2019-01-01",:]

In [216]:
print(X.shape)
print(Y.shape)

(187658, 7)
(187658, 2)


In [217]:
X.groupby([column for column in X.columns if column[0:6] == "target"]).count().reset_index().sort_values("fecha_expensa", ascending=False)

,target_lag_1,target_lag_2,target_lag_3,target_lag_4,target_lag_5,target_lag_6,fecha_expensa
0,0,0,0,0,0,0,47915
2237,3,3,3,3,3,3,12326
826,1,1,1,1,1,1,10357
2,0,0,0,0,0,2,4155
8,0,0,0,0,2,0,3160
1179,2,0,0,0,0,0,3032
28,0,0,0,2,0,0,2664
318,0,2,0,0,0,0,2650
96,0,0,2,0,0,0,2475
1590,2,2,2,2,2,2,2324


In [228]:
target_columns2 = [column for column in X.columns if column[0:6] == "target"]
X_train = X.loc[X.fecha_expensa < '2018-09-01', target_columns2]
X_test = X.loc[X.fecha_expensa >= '2018-09-01', target_columns2]
Y_train = Y.loc[Y.fecha_expensa < '2018-09-01', "target"]
Y_test =Y.loc[Y.fecha_expensa >= '2018-09-01', "target"]
Y_test_2 = Y.loc[Y.fecha_expensa >= '2018-09-01', "target"]
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(139028, 6) (48630, 6)
(139028,) (48630,)


In [229]:
X_train = X_train.iloc[28:,:]
X_test = X_test.iloc[:-30,:]
Y_train = Y_train.iloc[28:]
Y_test = Y_test.iloc[:-30]
Y_test_2 = Y_test_2.iloc[:-30]

In [230]:
# /X_train = pd.get_dummies(X_train).values.reshape(X_train.shape[0],12,4)
# X_test = pd.get_dummies(X_test).values.reshape(X_test.shape[0],12,4)

In [231]:
X_train = np.array(X_train.values).reshape(*X_train.shape)
X_test = np.array(X_test.values).reshape(*X_test.shape)

In [232]:
Y_train = np.array(pd.get_dummies(Y_train)[[0,1,2,3]].values).reshape(Y_train.shape[0],4)
Y_test = np.array(pd.get_dummies(Y_test)[[0,1,2,3]].values).reshape(Y_test.shape[0],4)

In [233]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape, Y_test_2.shape)

(139000, 6) (48600, 6)
(139000, 4) (48600, 4) (48600,)


In [237]:
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.layers import LSTM, Embedding

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=4, output_dim=4, input_length=1, batch_input_shape=(100,6)))
lstm_model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, stateful=True, kernel_initializer='random_uniform'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(20,activation='relu'))
lstm_model.add(Dense(4,activation='sigmoid'))
lstm_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [238]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (100, 1, 4)               16        
_________________________________________________________________
lstm_18 (LSTM)               (100, 100)                42000     
_________________________________________________________________
dropout_15 (Dropout)         (100, 100)                0         
_________________________________________________________________
dense_29 (Dense)             (100, 20)                 2020      
_________________________________________________________________
dense_30 (Dense)             (100, 4)                  84        
Total params: 44,120
Trainable params: 44,120
Non-trainable params: 0
_________________________________________________________________


In [239]:
lstm_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=40, verbose=2, batch_size=100,
                    shuffle=False)

Train on 139000 samples, validate on 48600 samples
Epoch 1/40
 - 8s - loss: 1.0052 - val_loss: 0.9251
Epoch 2/40
 - 7s - loss: 0.8474 - val_loss: 0.8300
Epoch 3/40
 - 7s - loss: 0.8057 - val_loss: 0.8066
Epoch 4/40
 - 7s - loss: 0.7859 - val_loss: 0.8001
Epoch 5/40
 - 7s - loss: 0.7754 - val_loss: 0.7977
Epoch 6/40
 - 7s - loss: 0.7709 - val_loss: 0.7955
Epoch 7/40
 - 7s - loss: 0.7690 - val_loss: 0.7934
Epoch 8/40
 - 10s - loss: 0.7668 - val_loss: 0.7922
Epoch 9/40
 - 9s - loss: 0.7652 - val_loss: 0.7909
Epoch 10/40
 - 9s - loss: 0.7647 - val_loss: 0.7913
Epoch 11/40
 - 9s - loss: 0.7639 - val_loss: 0.7895
Epoch 12/40
 - 9s - loss: 0.7630 - val_loss: 0.7891
Epoch 13/40
 - 8s - loss: 0.7636 - val_loss: 0.7899
Epoch 14/40
 - 9s - loss: 0.7631 - val_loss: 0.7894
Epoch 15/40
 - 9s - loss: 0.7620 - val_loss: 0.7891
Epoch 16/40
 - 9s - loss: 0.7617 - val_loss: 0.7885
Epoch 17/40
 - 8s - loss: 0.7602 - val_loss: 0.7890
Epoch 18/40
 - 8s - loss: 0.7593 - val_loss: 0.7879
Epoch 19/40
 - 9s - l

In [240]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
confusion_matrix(Y_test_2, lstm_model.predict_classes(X_test, batch_size=100))

array([[25907,   119,    37,  2053],
       [  461,  3821,     8,   410],
       [ 5528,    65,    48,  1430],
       [ 3601,    55,    25,  5032]], dtype=int64)

In [203]:
pred = lstm_model.predict_classes(X_test, batch_size=100)

In [204]:
confusion_matrix(Y_test_2, pred)

array([[18672,    87,     3,  1373],
       [  414,  2177,     1,   374],
       [ 4030,    61,     2,  1224],
       [ 2200,    35,     0,  3247]], dtype=int64)